In [31]:
"""
now, let's compute the equilibrium with local supply chains
"""
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
import scipy.optimize

rho=0.03
eta=0.4
theta=0.2
gamma=1.5
L_u=0.3
lambda_u=0.5
lambda_s=0.25

#first, we define the cost function
def s_star(w_u, w_s):
    temp1=(w_s/lambda_s-w_u/lambda_u)*lambda_s
    temp2=(lambda_u-lambda_s)*w_u/lambda_u
    if temp1<=temp2:
        return 0
    elif temp1>=temp2*np.exp(lambda_u):
        return 1
    else:
        return np.log(temp1/temp2)/lambda_u
def lu1(sstar):
    return (np.exp((lambda_u-lambda_s)*sstar)-np.exp(-lambda_s*sstar))*np.exp(lambda_s)/lambda_u
def ls1(sstar):
    return (np.exp(-lambda_s*sstar)-np.exp(-lambda_s))*np.exp(lambda_s)/lambda_s
def c(w_u, w_s):
    sstar=s_star(w_u, w_s)
    return w_u*lu1(sstar)+w_s*ls1(sstar)

#then, we express profit, research intensity, and north-leading firm proportion as functions of (w, w_u, w_s)
def pi_n(w, w_u, w_s):
    return 1-c(w_u, w_s)/(gamma*c(w, 100))
def pi_s(w, w_u, w_s):
    return 1-c(w, 10)/c(w_u, w_s)
def n_imi(w, w_u, w_s):
    return (pi_n(w, w_u, w_s)*eta/w_s-rho)/theta
def n_inn(w, w_u, w_s):
    return (pi_s(w, w_u, w_s)*theta/w-rho)/eta
def xi_n(w, w_u, w_s):
    return eta*n_inn(w, w_u, w_s)/(eta*n_inn(w, w_u, w_s)+theta*n_imi(w, w_u, w_s))

#third, we express the excess demand for three kinds of labors as functions of (w, w_u, w_s)
def southlaborexcessdemand(w, w_u, w_s):
    return xi_n(w, w_u, w_s)*n_imi(w, w_u, w_s)+(1-xi_n(w, w_u, w_s))*lu1(s_star(w, 100))/c(w_u, w_s)-1
def northunskillexcessdemand(w, w_u, w_s):
    return xi_n(w, w_u, w_s)*lu1(s_star(w_u, w_s))/(gamma*c(w, 100))-L_u
def northskillexcessdemand(w, w_u, w_s):
    return (1-xi_n(w, w_u, w_s))*n_inn(w, w_u, w_s)+xi_n(w, w_u, w_s)*ls1(s_star(w_u, w_s))/(gamma*c(w, 100))-(1-L_u)

#finally, we minimize the quadratic sum of those excess demands
def tol(x):
    return southlaborexcessdemand(x[0], x[1], x[2])**2+northunskillexcessdemand(x[0], x[1], x[2])**2+northskillexcessdemand(x[0], x[1], x[2])**2
lscwage=scipy.optimize.minimize(tol, (0.5, 0.8, 2), constraints=({'type': 'ineq', 'fun': lambda x: x[1]-x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: x[2]-x[1]},
                                                         {'type': 'ineq', 'fun': lambda x: x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: x[1]},
                                                         {'type': 'ineq', 'fun': lambda x: x[2]},
                                                         {'type': 'ineq', 'fun': lambda x: 1-x[0]}))
print("equilibrium under local supply chains", "\n",
     "south (unskilled) wage ", lscwage.x[0], "\n",
     "north unskilled wage ", lscwage.x[1], "\n",
     "north skilled wage ", lscwage.x[2], "\n",
     "north innovate intensity ", n_inn(lscwage.x[0], lscwage.x[1], lscwage.x[2]), "\n",
     "south imitation intensity ", n_imi(lscwage.x[0], lscwage.x[1], lscwage.x[2]), "\n",
     "north industrial share ", xi_n(lscwage.x[0], lscwage.x[1], lscwage.x[2]), "\n",
     "south industrial share ", 1-xi_n(lscwage.x[0], lscwage.x[1], lscwage.x[2]))

equilibrium under local supply chains 
 south (unskilled) wage  0.384854850719 
 north unskilled wage  0.535813132901 
 north skilled wage  0.578528566639 
 north innovate intensity  0.224862150156 
 south imitation intensity  0.310793136637 
 north industrial share  0.591339893686 
 south industrial share  0.408660106314


In [32]:
#check the steady-state condition
temp1=eta*n_inn(lscwage.x[0], lscwage.x[1], lscwage.x[2])*(1-xi_n(lscwage.x[0], lscwage.x[1], lscwage.x[2]))
temp2=theta*n_imi(lscwage.x[0], lscwage.x[1], lscwage.x[2])*(xi_n(lscwage.x[0], lscwage.x[1], lscwage.x[2]))
temp1-temp2

-6.9388939039072284e-18